Tensorflow is very useful to create, manage our models for deep learning, today we gonna test Tensorflow Server serving our models over the web and make it available like a Rest API

Importing the libraries

In [2]:
import tensorflow as tf
import numpy as np
import os

Creating a dataset

In [3]:
X = np.arange(-10.0, 10.0, 1e-2)
np.random.shuffle(X)
y = 2 * X + 1

train_end = int(0.6 * len(X))
test_start = int(0.8 * len(X))

X_train, y_train = X[:train_end], y[:train_end]
X_test, y_test = X[test_start:], y[test_start:]
X_val, y_val = X[train_end:test_start], y[train_end:test_start] 

Now let's create our model 

In [4]:
tf.keras.backend.clear_session()
linear_model = tf.keras.models.Sequential([
                                        tf.keras.layers.Dense(units=1, input_shape=[1], name='Single')
                                        ])
linear_model.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.mean_squared_error)
linear_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Single (Dense)              (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


Now let's train our model

In [5]:
linear_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20)
print(linear_model.evaluate(X_test, y_test, verbose=0))
print(round(linear_model.predict([7.443]).tolist()[0][0], 4))    # expected: 15.886

Epoch 1/20
38/38 [==============================] - 1s 6ms/step - loss: 9.6527 - val_loss: 0.1864
Epoch 2/20
38/38 [==============================] - 0s 2ms/step - loss: 0.0999 - val_loss: 0.0404
Epoch 3/20
38/38 [==============================] - 0s 2ms/step - loss: 0.0215 - val_loss: 0.0094
Epoch 4/20
38/38 [==============================] - 0s 3ms/step - loss: 0.0046 - val_loss: 0.0019
Epoch 5/20
38/38 [==============================] - 0s 3ms/step - loss: 0.0010 - val_loss: 4.2211e-04
Epoch 6/20
38/38 [==============================] - 0s 4ms/step - loss: 2.1787e-04 - val_loss: 9.0779e-05
Epoch 7/20
38/38 [==============================] - 0s 7ms/step - loss: 4.6396e-05 - val_loss: 2.0395e-05
Epoch 8/20
38/38 [==============================] - 0s 5ms/step - loss: 9.9769e-06 - val_loss: 5.2215e-06
Epoch 9/20
38/38 [==============================] - 0s 5ms/step - loss: 2.1868e-06 - val_loss: 8.7347e-07
Epoch 10/20
38/38 [==============================] - 0s 2ms/step - loss: 4.6122e-0

Finally let's save our model locally

In [6]:
export_path = 'linear_model/1/'
tf.saved_model.save(linear_model, export_path)

INFO:tensorflow:Assets written to: linear_model/1/assets


INFO:tensorflow:Assets written to: linear_model/1/assets


Next part gonna be preparing our docker container to host our model and make it acessible by a RestAPI Call